In [1]:
from pulp import *
from itertools import combinations_with_replacement

In [2]:
flyers_demand = {
    "A" : 68000,
    "B" : 14000,
    "C" : 16000,
    "D" : 12000,
    "F" : 12000,
}

prepairing_costs = 500
one_sheet_costs = 0.6
num_per_sheet = 9
min_print = 200
max_print = 50000

In [3]:
def flyersTotal(kombo, ftype, num):
    return kombo.count(ftype)*num

In [4]:
prob = LpProblem("Impose_optimization", LpMinimize)

In [5]:
flyer_types = list(flyers_demand.keys())
plate_combinations =[i for i in combinations_with_replacement(flyer_types, num_per_sheet)]

In [6]:
len(plate_combinations)

715

In [7]:
for i in plate_combinations:
    print(flyersTotal(i, 'C', 1),end="-")

0-0-1-0-0-0-1-0-0-2-1-1-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-4-3-3-2-2-2-1-1-1-1-0-0-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-4-3-3-2-2-2-1-1-1-1-0-0-0-0-0-5-4-4-3-3-3-2-2-2-2-1-1-1-1-1-0-0-0-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-4-3-3-2-2-2-1-1-1-1-0-0-0-0-0-5-4-4-3-3-3-2-2-2-2-1-1-1-1-1-0-0-0-0-0-0-6-5-5-4-4-4-3-3-3-3-2-2-2-2-2-1-1-1-1-1-1-0-0-0-0-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-4-3-3-2-2-2-1-1-1-1-0-0-0-0-0-5-4-4-3-3-3-2-2-2-2-1-1-1-1-1-0-0-0-0-0-0-6-5-5-4-4-4-3-3-3-3-2-2-2-2-2-1-1-1-1-1-1-0-0-0-0-0-0-0-7-6-6-5-5-5-4-4-4-4-3-3-3-3-3-2-2-2-2-2-2-1-1-1-1-1-1-1-0-0-0-0-0-0-0-0-0-1-0-0-2-1-1-0-0-0-3-2-2-1-1-1-0-0-0-0-4-3-3-2-2-2-1-1-1-1-0-0-0-0-0-5-4-4-3-3-3-2-2-2-2-1-1-1-1-1-0-0-0-0-0-0-6-5-5-4-4-4-3-3-3-3-2-2-2-2-2-1-1-1-1-1-1-0-0-0-0-0-0-0-7-6-6-5-5-5-4-4-4-4-3-3-3-3-3-2-2-2-2-2-2-1-1-1-1-1-1-1-0-0-0-0-0-0-0-0-8-7-7-6-6-6-5-5-5-5-4-4-4-4-4-3-3-3-3-3-3-2-2-2-2-2-2-2-1-1-1-1-1-1-1-1-0-0-0-0-0-0-0-0-0-0-1-0-0-2-

In [ ]:
#prob += LpVariable("uziti_desky", plate_combinantions, cat='Binary')

In [8]:
pouzite = pulp.LpVariable.dicts('pouzita_kombinace', plate_combinations, lowBound=0, upBound=1, cat = pulp.LpInteger)
mnozstvi = pulp.LpVariable.dicts('archu_z_kombinace', plate_combinations, lowBound=0, cat = pulp.LpInteger)

In [9]:
#tady se pokusim pridat expresi na minimalizaci
#seating_model += sum([happiness(table) * x[table] for table in possible_tables])
#
prob += lpSum([pouzite[plate] * prepairing_costs for plate in plate_combinations] #cena desky
              +
              [mnozstvi[plate] * one_sheet_costs for plate in plate_combinations]) 

In [ ]:
#lin = ['a','b','c']
#lin_var = [LpVariable(linvar, lowBound=0, upBound=9) for linvar in lin]
#f=LpAffineExpression([(lin_var[0],5),(lin_var[1],2)])

In [10]:
for key, value in flyers_demand.items():
    #toto je pridani constraints
    # prida podminu element je pomer uzitku na archu, dale klic slouzi k zapocteni vysledku z archu, pouzite==tistene mnozstvi archu
    prob += lpSum(flyersTotal(element,key,mnozstvi[element]) for element in plate_combinations) >= value
    
for plate in plate_combinations:
    #velmi dulezite zapoji to pouzitost nebo nepouzitost do hry, tj zajisti to, ze pokud je nejake mnozstvi
    #mnozstvi vetsi nez 0 znamena, ze musi byt sepnuta pouzitelnost jinak
    #pomaha hlavne ta druha podminka nebot pokud optimalizator zada mnozstvi na nepouzite desce
    #znamena to ze x >= max_print * 0 tj. x>=0 coz neni splneno
    prob += mnozstvi[plate] >= min_print * pouzite[plate]
    prob += mnozstvi[plate] <= max_print * pouzite[plate]

In [ ]:
#print(prob)

In [ ]:
import time
start_time = time.time()
prob.solve()
print(f"vysledek:{prob.status} za cas {time.time()-start_time}")

In [57]:
import time
start_time = time.time()
prob.solve(solvers.COIN_CMD(threads=4,msg=1,fracGap = 0.01))
print(f"vysledek:{prob.status} za cas {time.time()-start_time}")

vysledek:1 za cas 1586.1543395519257


In [ ]:
import time
start_time = time.time()
prob.solve(solvers.GLPK_CMD())
print(f"vysledek:{prob.status} za cas {time.time()-start_time}")

In [ ]:
import time
start_time = time.time()
prob.solve(solvers.CPLEX_PY())
print(f"vysledek:{prob.status} za cas {time.time()-start_time}")

In [ ]:
pulp.LpStatus[prob.status]

In [59]:
#optimalne potrebuji situaci dve desky cca 3.100ks
print('pouzite   kombinace \t\t mnozstvi')
for plate in plate_combinations:
    if (pouzite[plate].value() > 0):
        print(f'{pouzite[plate].value()} \t {plate} \t {mnozstvi[plate].value()}')

pouzite   kombinace 		 mnozstvi
1.0 	 ('A', 'A', 'A', 'A', 'A', 'B', 'C', 'D', 'F') 	 12728.0
1.0 	 ('A', 'A', 'A', 'A', 'B', 'B', 'C', 'C', 'C') 	 1091.0


In [60]:
def countTotalPrice(prepairNum):
    #cena
    total_price = 0
    for plate in plate_combinations:
        if (pouzite[plate].value() > 0):
           total_price += one_sheet_costs * mnozstvi[plate].value() + prepairing_costs
    return total_price

In [69]:
♥+1

SyntaxError: invalid character in identifier (<ipython-input-69-064f536700fc>, line 1)

In [61]:
countTotalPrice()

9291.4

In [62]:
import copy
pocty = copy.deepcopy(flyers_demand)

In [63]:
for plate in plate_combinations:
    if (pouzite[plate].value() > 0):
        for fl in pocty.keys():
            pocty[fl] -= flyersTotal(plate, fl, mnozstvi[plate].value())

In [64]:
import numpy as np

In [65]:
#kolik archu je navic
nppoc = np.array(list(pocty.values()))
nppoc.sum()/9

-263.44444444444446

In [51]:
flyers_demand

{'A': 68000, 'B': 14000, 'C': 16000, 'D': 12000, 'F': 12000}

In [53]:
for plate in plate_combinations:
    if (pouzite[plate].value() > 0):
        print(mnozstvi[plate].value())

13177.0
706.0


In [66]:
50/4

12.5